<a href="https://colab.research.google.com/github/yupAyush/BrainTumourDetection/blob/main/Braintumour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving kaggle.json to kaggle (1).json
User uploaded file "kaggle (1).json" with length 65 bytes


In [ ]:
!kaggle  datasets download -d sartajbhuvaji/brain-tumor-classification-mri


Dataset URL: https://www.kaggle.com/datasets/sartajbhuvaji/brain-tumor-classification-mri
License(s): MIT
brain-tumor-classification-mri.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip /content/brain-tumor-classification-mri.zip

Archive:  /content/brain-tumor-classification-mri.zip
replace Testing/glioma_tumor/image(1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import keras
from keras.models import Sequential
from keras.layers  import Conv2D,Flatten,Dense,MaxPool2D,Dropout
from sklearn.metrics import accuracy_score

In [ ]:
import ipywidgets as widgets
import io
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf

import numpy as np
import pandas as pd

In [ ]:
import os
import cv2
x_train=[]
y_train=[]
imgs=150
labels=['glioma_tumor','meningioma_tumor','no_tumor','pituitary_tumor']
for i in labels:
  folderPath=os.path.join('/content/Training',i)
  for j in os.listdir(folderPath):
    image= cv2.imread(os.path.join(folderPath,j))
    image=cv2.resize(image,(imgs,imgs))
    x_train.append(image)
    y_train.append(i)


for i in labels:
  folderPath=os.path.join('/content/Testing',i)
  for j in os.listdir(folderPath):
    image= cv2.imread(os.path.join(folderPath,j))
    image=cv2.resize(image,(imgs,imgs))
    x_train.append(image)
    y_train.append(i)

x_train =np.array(x_train)
y_train=np.array(y_train)

In [ ]:
x_train,y_train = shuffle(x_train,y_train,random_state=101)



In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_train,y_train,test_size=0.1,random_state=101)

In [ ]:
y_train_new=[]
for  i in y_train:
  y_train_new.append(labels.index(i))

y_train=y_train_new
y_train=tf.keras.utils.to_categorical(y_train)


y_test_new=[]
for i in y_test:
  y_test_new.append(labels.index(i))

y_test=y_test_new
y_test=tf.keras.utils.to_categorical(y_test)

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation = 'relu',input_shape=(150,150,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(Dropout(0.3))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(512,activation = 'relu'))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(4,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['acc'])

In [ ]:
history=model.fit(x_train,y_train ,epochs=20,validation_split=0.1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
model.save('brainTumourDetection.h5')

In [ ]:
acc =history.history['acc']
val_acc= history.history['val_acc']
epochs =range(len(acc))
fig =plt.figure(figsize=(14,7))
plt.plot(epochs,acc,'r',label=['Training_accuracy'])
plt.plot(epochs,val_acc,'b',label=['validation_accuracy'])
plt.legend(loc='upper left')
plt.show()

In [ ]:
loss =history.history['loss']
val_loss= history.history['val_loss']
epochs =range(len(loss))
fig =plt.figure(figsize=(14,7))
plt.plot(epochs,loss,'r',label=['Training_loss'])
plt.plot(epochs,val_loss,'b',label=['validation_loss'])
plt.legend(loc='upper left')
plt.show()

In [ ]:
img=cv2.imread('/content/Testing/glioma_tumor/image(100).jpg')
img=cv2.resize(img,(150,150))
img_array=np.array(img)
img_array=img_array.reshape(1,150,150,3)
img_array.shape


In [ ]:
from tensorflow.keras.preprocessing import image
img=image.load_img('//content/Testing/glioma_tumor/image(100).jpg')
plt.imshow(img,interpolation='nearest')
plt.show()


In [ ]:
a=model.predict(img_array)
ind=np.argmax(a,axis=1)
if ind==0:
  print("glioma_tumor")
elif ind==1:
   print("meningioma_tumor")
elif ind==2:
  pass

  print("no_tumor")
else:

  print("pitutary_tumor")
